<a href="https://colab.research.google.com/github/HanselWilfred/WeatherWise-Hansel-Wilfred/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧪 Download packages
!pip install fetch-my-weather
!pip install requests matplotlib pyinputplus







## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather

# import those packages

## 🌤️ Weather Data Functions

In [ ]:
# Define weather data
import requests, re

def get_weather_data(loc, forecast_days=5):
    """
    Given a location name or PIN code, it fetches up to 5 days of weather data
    using Open-Meteo APIs. Returns a dictionary with location, current temp,
    and daily forecast (date, min/max temperature, rain chance).
    """
    try:
        # Ensuring forecast_days is an integer between 1 and 5
        days = max(1, min(5, int(forecast_days or 5)))

        # Removing ambiguous words for better geocoding (like "today", "tomorrow", etc.)
        loc = re.sub(r"\b(tomorrow|today|day after|in\s+\d+\s+days)\b", "", str(loc), flags=re.I)
        # Removing excess whitespace and strip spaces
        loc = re.sub(r"\s+", " ", loc).strip()
        if not loc:  # If location is empty after cleaning, return empty dict
            return {}

        # 1. Geocoding: convert location name to coordinates
        g = requests.get(
            "https://geocoding-api.open-meteo.com/v1/search",
            params={"name": loc, "count":1, "language": "en"},
            timeout=10
        )
        g.raise_for_status()  # Raise HTTPError if something went wrong

        # Get the first result from geocoding
        hits = (g.json().get("results") or [])
        if not hits:
            return {}  # No valid geocoding result found

        # Extract coordinates and official city name
        lat, lon = hits[0]["latitude"], hits[0]["longitude"]
        city = hits[0].get("name", loc)

        # 2. Weather Forecast API call for daily temperatures and rain probability
        f = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={
                "latitude": lat,
                "longitude": lon,
                "timezone": "auto",
                "current_weather": True,
                "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_max"
            },
            timeout=10
        )
        f.raise_for_status()  # Raise HTTPError if something went wrong

        # Gather daily weather components from the response
        d = f.json().get("daily", {})
        dates = d.get("time", [])
        tmax = d.get("temperature_2m_max", [])
        tmin = d.get("temperature_2m_min", [])
        rain = d.get("precipitation_probability_max", [])
        n = min(days, len(dates))  # Limit to available days/data

        # Assemble the daily forecast as a list of dicts, each for one day
        fc = [{"date": dates[i], "max_temp": tmax[i], "min_temp": tmin[i], "rain_chance": rain[i]} for i in range(n)]

        # Return the full weather result for the location
        return {
            "location": city,
            "current_temp": f.json().get("current_weather", {}).get("temperature"),
            "forecast": fc
        }

    except Exception:
        # On error (network failure, bad data, etc.), return empty dict
        return {}





## 📊 Visualisation Functions

In [ ]:
# Visualise Data 1
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from datetime import datetime

def matplotlib_3d_temperature(weather_data):
    """
    Plots min and max daily temperatures in 3D using Matplotlib.
    X axis = dates, Y axis = temp type (Min or Max), Z axis = temperature value.
    """
    # Checking for valid weather data
    if not weather_data or "error" in weather_data or not weather_data.get("forecast"):
        print("No data to visualise.")
        return

    # Extracting date, min temp, max temp from forecast
    fc = weather_data['forecast']
    dates = [datetime.strptime(d['date'], "%Y-%m-%d") for d in fc]
    tmin = [d['min_temp'] for d in fc]
    tmax = [d['max_temp'] for d in fc]

    # Creating a new 3D figure and axis
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

    # Assigning numeric values for the date axis (for plotting)
    date_nums = np.arange(len(dates))

    # Plotting min temp as a blue line/markers in 3D
    ax.plot(date_nums, tmin, zs=0, zdir='y', label='Min Temp', color='blue', marker='o')

    # Plotting max temp as a red line/markers in 3D
    ax.plot(date_nums, tmax, zs=1, zdir='y', label='Max Temp', color='red', marker='o')

    # Customizing axes and labels for clarity
    ax.set_xlabel('Date')
    ax.set_ylabel('Temp Type')
    ax.set_zlabel('Temperature (°C)')
    ax.set_xticks(date_nums)
    ax.set_xticklabels([d.strftime("%d %b") for d in dates])
    ax.set_yticks([0,1])
    ax.set_yticklabels(['Min Temp', 'Max Temp'])

    # Adding title and legend
    ax.set_title(f"3D Min/Max Temperature — {weather_data.get('location','Location')}")
    ax.legend()

    # Showing the 3D plot
    plt.show()






In [ ]:
# Visualise Data 2
def matplotlib_lollipop_rain(weather_data):
    # Minimal lollipop chart for daily rain chance visualization
    # Validate input data: checking if weather_data and forecast exist and no error key
    if not weather_data or "error" in weather_data or not weather_data.get("forecast"):
        print("No data to visualise.")
        return

    # Importing necessary libraries for plotting and date manipulation
    import matplotlib.pyplot as plt
    from datetime import datetime

    # Extracting forecast list from weather_data
    fc = weather_data['forecast']
    # Extracting rain chance values, converting to floats, defaulting to 0 if missing
    rain = [float(d.get('rain_chance', 0)) for d in fc]
    # Extracting and formatting the date strings into human-readable day-month format for x-axis
    dates = [datetime.strptime(d['date'], "%Y-%m-%d").strftime("%d %b") for d in fc]

    # Creating matplotlib figure and axis with size proportional to number of dates
    fig, ax = plt.subplots(figsize=(1.2*len(dates), 4))

    # Plotting vertical lines ("sticks") for each date with light blue color and transparency
    ax.vlines(range(len(dates)), [0], rain, color="#adcbe3", alpha=0.7, lw=5)
    # Plotting blue circular markers ("candies") at the top of each line
    ax.scatter(range(len(dates)), rain, color="#2574a9", s=70, zorder=2)

    # Annotating each marker with rain percentage label slightly above the marker
    for i, val in enumerate(rain):
        ax.text(i, val+4, f"{int(val)}%", ha="center", va="bottom", fontsize=10)

    # Setting x-axis ticks and labels with formatted dates
    ax.set_xticks(range(len(dates)))
    ax.set_xticklabels(dates)

    # Labelling the y-axis to clarify this is chance of rain percentage
    ax.set_ylabel("Chance of Rain (%)")
    # Setting the y-axis limit max to either 100 or slightly higher than max rain value
    ax.set_ylim(0, max(100, max(rain)+10))
    # Setting the main plot title including location name from weather_data
    ax.set_title(f"Rain Chance — {weather_data.get('location','Location')}")
    # Setting a very light background color for minimal contrast and clean look
    ax.set_facecolor("#f7f8fa")

    # Adjusting layout to fit labels and elements properly without overlap
    plt.tight_layout()
    # Displaying the final lollipop plot
    plt.show()





## 🤖 Natural Language Processing

In [ ]:
# NLP + Response (with formatted dates)
import re
from datetime import datetime

def parse_weather_question(q):
    q = (q or "").lower().strip()
    # Determining which day user asks about: today=0, tomorrow=1, day after=2, or specific days offset
    day = 2 if ("day after tomorrow" in q or "day after" in q) else 1 if "tomorrow" in q else 0
    m = re.search(r"\bin\s+(\d+)\s+day(s)?\b", q)
    if m:
        try: day = max(0, min(4, int(m.group(1))))
        except: pass
    # Determining weather attribute: rain, temperature, or general
    attr = "rain" if any(w in q for w in ["rain", "rainy", "umbrella", "drizzle", "shower"]) else "temperature" if any(w in q for w in ["temp", "hot", "cold", "warm", "cool", "chill"]) else "general"
    # Searching for PIN/ZIP code
    pin = re.search(r"\b\d{3,6}\b", q)
    if pin: return {"location": pin.group(0), "day": day, "attribute": attr}
    # Searching for location name, excluding temporal words
    m = re.search(r"\bin\s+([a-z0-9\s\-]+?)(?=(?:\?|\.|,|$|\s+tomorrow|\s+today|\s+day after|\s+in\s+\d+\s+day[s]?))", q)
    loc = None
    if m:
        loc = re.sub(r"\b(tomorrow|today|day after tomorrow|day after|in\s+\d+\s+day[s]?)\b", "", m.group(1), flags=re.I).strip().title()
        if not loc: loc = None
    return {"location": loc, "day": day, "attribute": attr}


def generate_weather_response(parsed_question, data):
    loc, day, attr = parsed_question.get("location"), parsed_question.get("day",0), parsed_question.get("attribute","general")
    if not loc:  # Location missing
        return "Please tell me a city or PIN/ZIP (e.g., 'weather in Perth')."
    if not data or "error" in data:  # Weather API error
        return data.get("error", "Sorry, I couldn't get the weather right now.")
    fc = data.get("forecast") or []
    if not fc:  # No forecast data available
        return "No forecast available right now."
    i = min(day, len(fc)-1)
    d = fc[i]
    city = data.get("location", loc)
    date = d.get("date", "that day")
    try:
        date = datetime.strptime(date, "%Y-%m-%d").strftime("%d %b %Y")
    except:
        pass

    mx, mn = d.get("max_temp"), d.get("min_temp")
    rc = d.get("rain_chance")
    t = mx if mx is not None else mn

    def temp_phrase(x):
        if x is None: return "temperature unavailable"
        return "hot and sunny" if x >= 35 else "cool and pleasant" if x >= 15 else "cold"

    def rain_phrase(v):
        if v is None: return "No rain info."
        if v >= 50: return f"Yes, likely — rain {int(v)}%."
        if v >= 20: return f"Possible — rain {int(v)}%."
        return f"Unlikely — rain {int(v)}%."

    if attr == "rain":
        return f"In {city} on {date}: {rain_phrase(rc)}"

    msg = f"In {city} on {date}: {temp_phrase(t)}"
    if t is not None: msg += f" (around {round(t)}°C)"
    if rc is not None and rc >=30:
        msg += f", rain {int(rc)}%."
    else:
        msg += "."
    return msg


## 🧭 User Interface

In [ ]:
# UI (A):

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


def print_forecast(res):
    if not res or "error" in res:
        print(f"⚠️ {res.get('error','No data')}")
        return
    cur = res.get('current_temp')
    print(f"📍 {res['location']} | 🌡️ {cur}°C" if isinstance(cur, (int,float)) else "N/A")
    for f in res.get("forecast", []):
        d = f.get('date','')
        try:
            d = datetime.strptime(d, "%Y-%m-%d").strftime("%d %b")
        except:
            pass
        # Choosing temp for emoji determination
        temp = f.get("min_temp") if isinstance(f.get("min_temp"), (int,float)) else f.get("max_temp") if isinstance(f.get("max_temp"), (int,float)) else None
        emoji = "🔥" if temp and temp >= 35 else "🙂" if temp and temp >= 15 else "🥶"
        # Formatting temperature string
        temp_str = f"{f.get('min_temp'):.1f}–{f.get('max_temp'):.1f}°C" if all(isinstance(t, (int,float)) for t in [f.get("min_temp"), f.get("max_temp")]) else "temp N/A"
        # Annotating rain chance if significant
        rc = f.get("rain_chance")
        try:
            rcv = float(rc)
        except:
            rcv = None
        rain_note = f" (rain {int(rcv)}%)" if rcv and rcv >=30 else ""
        print(f"{d}: {temp_str} {emoji}{rain_note}")


def create_temperature_visualisation(res):
    if not res or "error" in res or not res.get("forecast"):
        print("No data")
        return
    fc = res["forecast"]
    dates = [datetime.strptime(r["date"], "%Y-%m-%d") for r in fc]
    tmin = [r.get("min_temp") for r in fc]
    tmax = [r.get("max_temp") for r in fc]

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection="3d")

    date_nums = np.arange(len(dates))
    # Plotting min and max temps as separate lines at fixed y positions (0 and 1)
    ax.plot(date_nums, [0]*len(dates), tmin, color="blue", marker="o", label="Min Temp")
    ax.plot(date_nums, [1]*len(dates), tmax, color="red", marker="o", label="Max Temp")

    # Labelling axes and ticks clearly
    ax.set_xlabel("Date")
    ax.set_ylabel("Temp Type")
    ax.set_zlabel("Temperature (°C)")
    ax.set_xticks(date_nums)
    ax.set_xticklabels([d.strftime("%d %b") for d in dates])
    ax.set_yticks([0,1])
    ax.set_yticklabels(["Min", "Max"])
    ax.set_title(f"3D Min/Max Temp — {res.get('location','Location')}")
    ax.legend()
    plt.show()



In [ ]:
# UI (B):

def create_precipitation_visualisation(res):
    # Checking for valid data
    if not res or "error" in res or not res.get("forecast"):
        print("No data")
        return

    fc = res["forecast"]
    xs = range(len(fc))  # X-axis numeric indices for days
    dates = [datetime.strptime(r["date"], "%Y-%m-%d").strftime("%d %b") for r in fc]  # Formatted date strings
    # Ensuring rain percentages are valid between 0 and 100
    rainp = [max(0, min(100, float(r.get("rain_chance",0)))) for r in fc]

    # Filled area below rain chance curve with light transparency
    plt.fill_between(xs, 0, rainp, alpha=0.15)
    # Line plot for rain chance trend
    plt.plot(xs, rainp, linewidth=2)

    # Vertical lines, markers, and labels for each rain chance point
    for x, v in zip(xs, rainp):
        plt.vlines(x, 0, v)
        plt.scatter(x, v, s=50)
        plt.text(x, v+2, f"{int(v)}%", ha="center", fontsize=8)

    # Chart title with location
    plt.title(f"Rain Chance — {res.get('location')}")
    plt.ylim(0, 100)  # Limits for y-axis percentage range
    plt.xticks(xs, dates)  # Date labels on x-axis
    plt.tight_layout()
    plt.show()


def create_both_visualisations(res):
    # Calling both temperature and precipitation visualiszations sequentially
    create_temperature_visualisation(res)
    create_precipitation_visualisation(res)




## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def run_app():
    print("🌦️ Welcome to WeatherWise 🌦️")
    print("Enter city name or PIN/ZIP.")
    while True:
        print("\nOptions:\n1. View Forecast\n2. Ask Question\n3. Visualise Data\n4. Exit")
        c = input("> ").strip()
        if c == "1":
            p = input("City or PIN/ZIP: ").strip()
            print_forecast(get_weather_data(p))
        elif c == "2":
            q = input("Ask about weather: ").strip()
            info = parse_weather_question(q)
            p = info.get("location")
            if not p:
                p = input("City or PIN/ZIP: ").strip()
                q = f"{q} in {p}"
                info = parse_weather_question(q) # Parse the updated query
            print(generate_weather_response(info, get_weather_data(info.get("location"))))
        elif c == "3":
            p = input("City to visualise: ").strip()
            res = get_weather_data(p)
            if not res or "error" in res:
                print(f"⚠️ {res.get('error','No data')}")
                continue

            print("1) Temperature  2) Rain  3) Both")
            v = input("> ").strip().lower()

            if v in ("1", "temp", "temperature"):
                create_temperature_visualisation(res)
            elif v in ("2", "lollipop", "stem", "rainmeter", "rainlollipop", "rain"):
                matplotlib_lollipop_rain(res)
            elif v in ("3", "both", "all"):
                create_temperature_visualisation(res)
                matplotlib_lollipop_rain(res)
            else:
                print("Choose 1, 2, or 3.")
        elif c == "4":
            print("Goodbye! ☀️")
            break
        else:
            print("Choose 1–4.")

## 🧪 Testing and Examples

In [ ]:
# Questions you can ask weatherwise
SAMPLE_QUESTIONS = [
    # Rain-only
    "Will it rain today?",
    "Is rain likely in Chennai tomorrow?",

    # Temperature-only
    "Is it cold tomorrow in Melbourne?",
    "Temperature in Adelaide?",
    "Tell me the weather in 6000?",

    # General feel
    "How is the weather?",
    "Show me the temperature in Brisbane in 2 days?",
    "Do I need an umbrella tomorrow in Perth?",
]

run_app()